In [1]:
import torch
import json 
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import warnings
warnings.filterwarnings('ignore')

/home/nice/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """
/home/nice/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
#T5 is a new transformer model from Google
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [3]:
df=pd.read_excel('/home/nice/Downloads/TASK.xlsx')
df.dropna(axis=1,inplace=True)
df[:5]

,Intoduction
0,Acnesol Gel is an antibiotic that fights bacte...
1,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...


In [5]:
text =df.Intoduction[1]
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

original text preprocessed: 
 Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus. It works by thinning and loosens mucus in the nose, windpipe and lungs and make it easier to cough out.Ambrodil Syrup should be taken with food. For better results, it is suggested to take it at the same time every day.  The dose and how often you take it depends on what you are taking it for. Your doctor will decide how much you need to improve your symptoms.  It is advised not to use it for more than 14 days without doctor consultation.The most common side effects of this medicine include vomiting, nausea, and stomach upset. Talk to your doctor if you are worried about side effects or they would not go away. Generally, it is advised not to take alcohol while on treatment.Before taking this medicine, tell your doctor if you have liver or kidney disease or if you have stomach problems. Your doctor should also know about all other medicines you are takin

In [6]:
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)




Summarized text: 
 Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus. it works by thinning and loosens mucous in the nose, windpipe and lungs and make it easier to cough out. the dose and how often you take it depends on what you are taking it for. your doctor will decide how much you need to improve your symptoms.


In [4]:
def summmarize(text):
    preprocess_text = text.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=100,
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output


In [ ]:
df['summmary']=df['Intoduction'].progress_apply(summmarize)
df.to_csv('df.csv')

In [3]:
result=pd.read_csv('/home/nice/Downloads/df.csv')
result.drop(columns='Unnamed: 0',inplace=True)

In [15]:
print(result.Intoduction[1])

Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus. It works by thinning and loosens mucus in the nose, windpipe and lungs and make it easier to cough out.Ambrodil Syrup should be taken with food. For better results, it is suggested to take it at the same time every day.  The dose and how often you take it depends on what you are taking it for. Your doctor will decide how much you need to improve your symptoms.  It is advised not to use it for more than 14 days without doctor consultation.The most common side effects of this medicine include vomiting, nausea, and stomach upset. Talk to your doctor if you are worried about side effects or they would not go away. Generally, it is advised not to take alcohol while on treatment.Before taking this medicine, tell your doctor if you have liver or kidney disease or if you have stomach problems. Your doctor should also know about all other medicines you are taking as many of these may make th

In [16]:
print(result.summmary[1])

Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus. it works by thinning and loosens mucous in the nose, windpipe and lungs and make it easier to cough out. the dose and how often you take it depends on what you are taking it for. your doctor will decide how much you need to improve your symptoms.
